### Importing the libraries

In [3]:
import numpy as np
import time as t
import cv2
import math
import pyautogui as p

### Main Code

In [4]:

# Capturing video through front camera(webcam)
cap=cv2.VideoCapture(0,cv2.CAP_DSHOW)

def nothing(x):
    pass

#Window Name where we adjust the visibility of our hand
cv2.namedWindow("Colour Adjustments",cv2.WINDOW_NORMAL)
cv2.resizeWindow("Colour Adjustments",(300,300))
cv2.createTrackbar("Thresh","Colour Adjustments",0,255,nothing)

#Create Colour Detection Track
cv2.createTrackbar("Lower_H","Colour Adjustments",0,255,nothing)
cv2.createTrackbar("Lower_S","Colour Adjustments",0,255,nothing)
cv2.createTrackbar("Lower_V","Colour Adjustments",0,255,nothing)
cv2.createTrackbar("Upper_H","Colour Adjustments",255,255,nothing)
cv2.createTrackbar("Upper_S","Colour Adjustments",255,255,nothing)
cv2.createTrackbar("Upper_V","Colour Adjustments",255,255,nothing)

while True:
    _,frame=cap.read()
    frame=cv2.flip(frame,2)
    frame=cv2.resize(frame,(500,400))
    
    #Creating the box where hand data is going to be read
    frame=cv2.rectangle(frame,(0,1),(250,400),(255,0,0),0)
    crop_image=frame[1:400,0:250]
    #NOTE:After resizing the shape of our frame is (400,500,3) ie. 400 rows,500 columns and 3 channels. Now since we have to
    #crop image vertically therefore no. of rows will remain same and no. of columns will get halved
    
    #Coverting BGR to HSV
    #Reason-HSV format is better suited for object detection as compared to RGB,BGR
    hsv=cv2.cvtColor(crop_image,cv2.COLOR_BGR2HSV)
    
    #Capturing the value of trackbar values/Detecting hand
    l_h=cv2.getTrackbarPos("Lower_H","Colour Adjustments")
    l_s=cv2.getTrackbarPos("Lower_S","Colour Adjustments")
    l_v=cv2.getTrackbarPos("Lower_V","Colour Adjustments")
    u_h=cv2.getTrackbarPos("Upper_H","Colour Adjustments")
    u_s=cv2.getTrackbarPos("Upper_S","Colour Adjustments")
    u_v=cv2.getTrackbarPos("Upper_V","Colour Adjustments")
    
    #creating bounds
    lower_bound=np.array([l_h,l_s,l_v])
    upper_bound=np.array([u_h,u_s,u_v])
    #Any colour will have a darkest pixel value and lightest pixel value , that is why the bounds have been created 
    
    #Creating Mask
    mask=cv2.inRange(hsv,lower_bound,upper_bound)
    #In the image hsv only show me those pixels whose colour range lie in between lower_bound(darkest pixel) and
    #upper_bound(lightest pixel)
    
    #Filtering mask with image
    filtre=cv2.bitwise_and(crop_image,crop_image,mask=mask)
    
    #inverting the pixels so that the image is white and background is black because this type of setting is ideal in 
    #detecting the contours
    mask1=cv2.bitwise_not(mask)
    
    #Storing the thresh value in a variable(not compulsory)
    m_g=cv2.getTrackbarPos("Thresh","Colour Adjustments")
    
    #Creating Threshold (Thresholding makes our image easier to analyze during image processing)
    ret,thresh=cv2.threshold(mask1,m_g,255,cv2.THRESH_BINARY)
    
    #Performing Dilation for better processing(handling noise)
    dilata=cv2.dilate(thresh,(3,3),iterations=6)
    
    #finding Contours 
    cnts,hier=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #try and except used for graceful termination of code
    try:
        #find Contours with maximum area
        cm=max(cnts,key=lambda x:cv2.contourArea(x))
        
        #Contour Approximation
        epsilon=0.0005*cv2.arcLength(cm,True)
        data=cv2.approxPolyDP(cm,epsilon,True)
        hull=cv2.convexHull(cm)
        
        #Drawing the contours
        cv2.drawContours(crop_image,[cm],-1,(50,50,150),2)
        cv2.drawContours(crop_image,[hull],-1,(0,255,0),2)
        
        #Finding Convexity defect (convexity defect is a cavity in an object (blob, contour) segmented out from an image)
        #That means an area that do not belong to the object but located inside of its outer boundary 
        hull=cv2.convexHull(cm,returnPoints=False)
        defects=cv2.convexityDefects(cm,hull)
        #https://theailearner.com/2020/11/09/convexity-defects-opencv/ (Explaining convexity defects)
        count_defects=0
        
        for i in range(defects.shape[0]):
            s,e,f,d=defects[i,0]
            start = tuple(cm[s][0])
            end = tuple(cm[e][0])
            far = tuple(cm[f][0])
            
            #https://medium.com/analytics-vidhya/hand-detection-and-finger-counting-using-opencv-python-5b594704eb08
            #A very good article explaining the math concept perfectly
            
            #Calculating the length of sides so that we can apply cosine rule
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            
            #Applying cosine rule
            angle = (math.acos((b**2 + c**2 - a**2) / (2*b*c))*180) / 3.14
            
            #Plotting the far points on the basis of angle
            if(angle<=50):
                count_defects+=1
                cv2.circle(crop_image,far,5,(255,255,255),-1)
            
        if(count_defects==0):
            cv2.putText(frame," ",(50,50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),2)
        
        elif(count_defects==1):
            p.press("space")
            cv2.putText(frame,"Play/Pause",(50,50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),2)
            
        elif(count_defects==2):
            p.press("up")
            cv2.putText(frame,"Volume UP",(50,50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),2)
            
        elif(count_defects==3):
            p.press("down")
            cv2.putText(frame,"Volume DOWN",(50,50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),2)
            
        elif(count_defects==4):
            p.press("right")
            cv2.putText(frame,"Forward",(50,50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),2)
            
    except:
        pass
    
    #Printing the results        
    cv2.imshow("Result",frame)
    cv2.imshow("Thresh",thresh)
    cv2.imshow("Filtre",filtre)
    #cv2.imshow("mask",mask)
    #cv2.imshow("mask1",mask1)
    if(cv2.waitKey(25) & 0xFF==ord('q')):
        break
cap.release()
cv2.destroyAllWindows()

In [7]:
#-----------------------------------------------------------------------------
#Step - 1  -Import Libraries and capture camera
#Step - 2  -Convert frames Into hsv
#Step - 3  -Track hand on color basis 
#Step - 4  -Create mask on the basis of color and filter actual color 
#Step - 5  -Invert pixel value and then enchance the result for better output
#Step - 6  -Find Contours for specific colored object
#Step - 7  -Find Max area contour and draw it on live feed
#Step - 8  -Find Convexity detect  for counting Values and Apply Cosin method
#Step - 9  -Bind hand gestures with keyboard keys.
#Step -10  -Enjoy your output

#-----------------Gesture Control Media Player---------------------------

#Step -1
import cv2
import numpy as np 
import math
import pyautogui as p
import time as t



#Read Camera
cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
def nothing(x):
    pass
#window name
cv2.namedWindow("Color Adjustments",cv2.WINDOW_NORMAL)
cv2.resizeWindow("Color Adjustments", (300, 300)) 
cv2.createTrackbar("Thresh", "Color Adjustments", 0, 255, nothing)

#COlor Detection Track

cv2.createTrackbar("Lower_H", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Lower_S", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Lower_V", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Upper_H", "Color Adjustments", 255, 255, nothing)
cv2.createTrackbar("Upper_S", "Color Adjustments", 255, 255, nothing)
cv2.createTrackbar("Upper_V", "Color Adjustments", 255, 255, nothing)


while True:
    _,frame = cap.read()
    frame = cv2.flip(frame,2)
    frame = cv2.resize(frame,(600,500))
    # Get hand data from the rectangle sub window
    cv2.rectangle(frame, (0,1), (300,500), (255, 0, 0), 0)
    crop_image = frame[1:500, 0:300]
    
    #Step -2
    hsv = cv2.cvtColor(crop_image, cv2.COLOR_BGR2HSV)
    #detecting hand
    l_h = cv2.getTrackbarPos("Lower_H", "Color Adjustments")
    l_s = cv2.getTrackbarPos("Lower_S", "Color Adjustments")
    l_v = cv2.getTrackbarPos("Lower_V", "Color Adjustments")

    u_h = cv2.getTrackbarPos("Upper_H", "Color Adjustments")
    u_s = cv2.getTrackbarPos("Upper_S", "Color Adjustments")
    u_v = cv2.getTrackbarPos("Upper_V", "Color Adjustments")
    #Step -3
    lower_bound = np.array([l_h, l_s, l_v])
    upper_bound = np.array([u_h, u_s, u_v])
    
    #Step - 4
    #Creating Mask
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    #filter mask with image
    filtr = cv2.bitwise_and(crop_image, crop_image, mask=mask)
    
    #Step - 5
    mask1  = cv2.bitwise_not(mask)
    m_g = cv2.getTrackbarPos("Thresh", "Color Adjustments") #getting track bar value
    ret,thresh = cv2.threshold(mask1,m_g,255,cv2.THRESH_BINARY)
    dilata = cv2.dilate(thresh,(3,3),iterations = 6)
    
    #Step -6
    #findcontour(img,contour_retrival_mode,method)
    cnts,hier = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    
    try:
        #print("try")
         #Step -7
         # Find contour with maximum area
        cm = max(cnts, key=lambda x: cv2.contourArea(x))
        #print("C==",cnts)
        epsilon = 0.0005*cv2.arcLength(cm,True)
        data= cv2.approxPolyDP(cm,epsilon,True)
    
        hull = cv2.convexHull(cm)
        
        cv2.drawContours(crop_image, [cm], -1, (50, 50, 150), 2)
        cv2.drawContours(crop_image, [hull], -1, (0, 255, 0), 2)
        
        #Step - 8
        # Find convexity defects
        hull = cv2.convexHull(cm, returnPoints=False)
        defects = cv2.convexityDefects(cm, hull)
        count_defects = 0
        #print("Area==",cv2.contourArea(hull) - cv2.contourArea(cm))
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
           
            start = tuple(cm[s][0])
            end = tuple(cm[e][0])
            far = tuple(cm[f][0])
            #Cosin Rule
            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14
            #print(angle)
            # if angle <= 50 draw a circle at the far point
            if angle <= 50:
                count_defects += 1
                cv2.circle(crop_image,far,5,[255,255,255],-1)
        
        print("count==",count_defects)
        
        #Step - 9 
        # Print number of fingers
        if count_defects == 0:
            
            cv2.putText(frame, " ", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)
        elif count_defects == 1:
            
            p.press("space")
            cv2.putText(frame, "Play/Pause", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 2:
            p.press("up")
            
            cv2.putText(frame, "Volume UP", (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 3:
            p.press("down")
            
            cv2.putText(frame, "Volume Down", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 4:
            p.press("right")
            
            cv2.putText(frame, "Forward", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        else:
            pass
           
    except:
        pass
    #step -10    
    cv2.imshow("Thresh", thresh)
    #cv2.imshow("mask==",mask)
    cv2.imshow("filter==",filtr)
    cv2.imshow("Result", frame)

    key = cv2.waitKey(1) &0xFF    
    if key == 27: 
        break
cap.release()
cv2.destroyAllWindows()

count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0


count== 3
count== 4
count== 2
count== 3
count== 3
count== 3
count== 4
count== 4
count== 4
count== 3
count== 2
count== 3
count== 1
count== 4
count== 4
count== 2
count== 4
count== 4
count== 0
count== 2
count== 3
count== 4
count== 0
count== 1
count== 2
count== 1
count== 1
count== 1
count== 1
count== 1
count== 1
count== 1
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0


count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 1
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0


count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 1
count== 0
count== 0
count== 0
count== 0
count== 1
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 1
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 1
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0


count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 1
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0


count== 1
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 1
count== 1
count== 1
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 1
count== 1
count== 0
count== 0
count== 0
count== 1
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 1
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 1
count== 0
count== 1
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
count== 0
